### Determine application type

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.12

In [3]:
using System;
using System.IO;
string directory = "../config";
string fullPath = Path.GetFullPath(directory);
Console.WriteLine(fullPath);

c:\Users\mrochon\source\repos\ai-samples\NotebookAI\config


Setup OpenAI client

In [6]:
#!import ../config/Settings.cs

using Azure;
using Azure.AI.OpenAI;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();

OpenAIClient client = new(new Uri(azureEndpoint), new AzureKeyCredential(apiKey));

var messages = new ChatCompletionsOptions()
{
    DeploymentName = model,
    Messages =
    {
        new ChatRequestSystemMessage(System.IO.File.ReadAllText("system.txt"))
    },
    MaxTokens = 100
};


Chat

In [7]:
do
{
    string? userMessage = await InteractiveKernel.GetInputAsync("System description:");
    if (string.IsNullOrEmpty(userMessage))
        break;
    messages.Messages.Add(new ChatRequestUserMessage(userMessage));
    using(StreamingResponse<StreamingChatCompletionsUpdate> response = await client.GetChatCompletionsStreamingAsync(messages))
    {
        var txt = new StringBuilder();
        await foreach (var choice in response)
        {
            if (choice.ContentUpdate != null)
            {
                var txt1 = choice.ContentUpdate;
                txt.Append(txt1);
                // if (choice.Completion.Length > 0 && choice.Completion[^1] != ' ')
                //     txt.Append(' ');
            }
        }
        Console.Write(txt);
        Console.WriteLine();
    }
} while (true);


Error: Command cancelled.